In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from passivbot import *
from vwap import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams['figure.figsize'] = [21, 13]
pd.set_option('precision', 10)

In [ ]:
user = 'ebnsub1'
commons, vwap, streamer = prepare_bot('binance', user)
vwap.stop_updater()
commons.stop_call_limiter()

In [ ]:
equities = {}
for c in vwap.all_coins_set:
    if type(vwap.balance[c]) == int:
        continue
    s_ = c + '/BTC'
    if c != 'BTC' and s_ not in vwap.cc.markets:
        continue
    equities[c] = vwap.balance[c]['equity'] / vwap.balance[c]['account_equity']
print('\n\n(coin equity) divided by (account equity in terms of coin)\n')
for e in sorted(equities.items(), key=lambda x: x[1]):
    print(f'{e[0]:7} {e[1]:.6f} {"liquidating" if e[0] in vwap.do_liquidate else ""}')


In [ ]:
# uncomment and run this to set emas to current price
'''
tickers = vwap.cc.fetch_tickers()
for sym in vwap.symbols:
    vwap.cm.last_price[sym] = tickers[sym]['last']
    vwap.cm.min_ema[sym] = tickers[sym]['last']   
    vwap.cm.max_ema[sym] = tickers[sym]['last']   
'''

In [ ]:
print('n days trade history in memory\n')
rm_ = [((vwap.my_trades[s][-1]['timestamp'] - vwap.my_trades[s][0]['timestamp']) / 1000 / 60 / 60 / 24, s) for s in vwap.symbols]
for e in sorted(rm_):
    print(f"{e[1]:13} {e[0]:.2f}")

In [ ]:
# set symbol for analysis
s = 'RVN/BTC'
c, q = s.split('/')

In [ ]:
vwap.update_balance()
vwap.init_my_trades(s)
#vwap.update_my_trades(s)
vwap.set_analysis(s, vwap.my_trades[s])
vwap.init_loan_history(c, 'borrow')
vwap.init_loan_history(c, 'repay')

In [ ]:
vwap.update_open_orders(s)
print(f'\ncurrent open orders for {s}\n')
for my_orders in [vwap.my_bids[s], vwap.my_asks[s]]:
    for o in my_orders:
        print({k: o[k] for k in ['symbol', 'side', 'amount', 'price']})


In [ ]:
events = sorted(vwap.my_trades[s] + vwap.borrow_history[c] + vwap.repay_history[c], key=lambda x: x['timestamp'])
do_print = False
n = 100
print(f'most recent {n} events')
for e in events[-n:]:
    line = f"{ts_to_date(e['timestamp'] / 1000)[:19]}  {e['side']:7} {e['amount']:.8f}"
    if 'price' in e:
        do_print = True
        line += f" {e['price']}"
    if do_print:
        print(line)

In [ ]:
for s_ in vwap.symbols:
#for s_ in [s]:
    vwap.set_ideal_orders(s_)
vwap.allocate_credit()
for c_ in [c, q]:
    for key in ['free', 'onhand', 'debt', 'borrowable']:
        print(f"{c_} {key:10} {vwap.balance[c_][key]}")
    print()
print('borrow', vwap.ideal_borrow[c])
print('repay ', vwap.ideal_repay[c])
print()
print('last_price', vwap.cm.last_price[s])
print('\nideal orders')
print('long_buy', vwap.ideal_long_buy[s])
print('shrt_sel', vwap.ideal_shrt_sel[s])
print('long_sel', vwap.ideal_long_sel[s])
print('shrt_buy', vwap.ideal_shrt_buy[s])
print('\nliqui_buy', vwap.ideal_liqui_buy[s])
print('liqui_sel', vwap.ideal_liqui_sel[s])

print()
print('eligible orders')
for order in vwap.eligible_entries + vwap.eligible_exits:
    if order['symbol'] == s:
        print(order)
print()
print('sum sel amounts', vwap.ideal_shrt_sel[s]['amount'] + vwap.ideal_long_sel[s]['amount'])
print('sum onhand', vwap.balance[c]['onhand'] + vwap.ideal_borrow[c])
print()
print('sum buys cost', (vwap.ideal_shrt_buy[s]['amount'] * vwap.ideal_shrt_buy[s]['price'] +
                        vwap.ideal_long_buy[s]['amount'] * vwap.ideal_long_buy[s]['price']))
now_millis = vwap.cc.milliseconds()
small_trade_cost = max(vwap.min_trade_costs[s],
                       vwap.balance[q]['account_equity'] * vwap.account_equity_pct_per_trade[s])
entry_delay_millis = (60 * 60 * 1000) / (vwap.account_equity_pct_per_hour[s] / small_trade_cost)
print('\nminutes delay between entries', round(entry_delay_millis / 1000 / 60, 2))
print('minutes since prev long entry',
      round((now_millis - vwap.my_trades_analyses[s]['last_long_entry_ts']) / 1000 / 60, 2))
print('minutes since prev shrt entry',
      round((now_millis - vwap.my_trades_analyses[s]['last_shrt_entry_ts']) / 1000 / 60, 2))



In [ ]:
# true size of positions
vwap.my_trades_analyses[s]

In [ ]:
# dataframe of all trades
n_days = vwap.settings['max_memory_span_days']
mts = vwap.fetch_my_trades(s)
mtdf = pd.DataFrame(mts).set_index('timestamp').drop(['id', 'order_id', 'type'], axis=1)
mtdf = mtdf[mtdf.index > (vwap.cc.milliseconds() - 1000 * 60 * 60 * 24 * n_days)]

In [ ]:
threshold = vwap.my_trades_analyses[s]['entry_exit_amount_threshold']
print('threshold', threshold)
big_trades = mtdf[mtdf.amount > threshold]
small_trades = mtdf[mtdf.amount <= threshold]
long_buys = small_trades[small_trades.side == 'buy']
shrt_sels = small_trades[small_trades.side == 'sell']
long_sels = big_trades[big_trades.side == 'sell']
shrt_buys = big_trades[big_trades.side == 'buy']

lbc = long_buys.cost.sum()
lba = long_buys.amount.sum()
lsc = long_sels.cost.sum()
lsa = long_sels.amount.sum()

ssc = shrt_sels.cost.sum()
ssa = shrt_sels.amount.sum()
sbc = shrt_buys.cost.sum()
sba = shrt_buys.amount.sum()

print('total quot vol', mtdf.cost.sum())
print('n_days', round((mtdf.index[-1] - mtdf.index[0]) / 1000 / 60 / 60 / 24, 2))
print()

if lba:
    print('long_buy_cost', round(lbc, 8), 'long_buy_vwap', round(lbc / lba, 10))
if lsa:
    print('long_sel_cost', round(lsc, 8), 'long_sel_vwap', round(lsc / lsa, 10))
if lba and lsa:
    print('long_sel_vwap / long_buy_vwap', round((lsc / lsa) / (lbc / lba), 10))

print()

if ssa:
    print('shrt_sel_cost', round(ssc, 8), 'shrt_sel_vwap', round(ssc / ssa, 10))
if sba:
    print('shrt_buy_cost', round(sbc, 8), 'shrt_buy_vwap', round(sbc / sba, 10))
if ssa and sba:
    print('shrt_sel_vwap / shrt_buy_vwap', round((ssc / ssa) / (sbc / sba), 10))

all_buys = mtdf[mtdf.side == 'buy']
all_sels = mtdf[mtdf.side == 'sell']
abc = all_buys.cost.sum()
aba = all_buys.amount.sum()
asc = all_sels.cost.sum()
asa = all_sels.amount.sum()

print()
print('all buy_cost', round(abc, 8), 'buy_vwap', round(abc / aba, 10))
print('all sel_cost', round(asc, 8), 'sel_vwap', round(asc / asa, 10))
print('all sel_vwap / buy_vwap', round((asc / asa) / (abc / aba), 10))

    


In [ ]:
# blue dots are long buys, red dots are long sells, red cross is current long sell price
long_buys.price.plot(style='b.-')
long_sels.price.plot(style='ro')
long_sel_price = vwap.my_trades_analyses[s]['long_sel_price']
if long_sel_price > 0.0:
    plt.plot([long_buys.index[-1]], [long_sel_price], 'r+')
    print(f'long_exit_price / current_price {long_sel_price / vwap.cm.last_price[s]:.4f}')


In [ ]:
# red dots are short sells, blue dots are short buys, blue cross is current short buy price
shrt_sels.price.plot(style='r.-')
shrt_buys.price.plot(style='bo')
shrt_buy_price = vwap.my_trades_analyses[s]['shrt_buy_price']
if shrt_buy_price > 0.0:
    plt.plot([shrt_sels.index[-1]], [shrt_buy_price], 'b+')
    print(f'current_price / shrt_exit_price {vwap.cm.last_price[s] / shrt_buy_price:.4f}')

In [ ]:
# all buys and sells in one
all_sels.price.plot(style='ro-')
all_buys.price.plot(style='bo-')

In [ ]:
tlc = {} # true long costs
tsc = {} # true shrt costs
long_vwaps = {}
shrt_vwaps = {}
for s_ in vwap.my_trades_analyses:
    tlc[s_] = vwap.my_trades_analyses[s_]['true_long_cost']
    tsc[s_] = vwap.my_trades_analyses[s_]['true_shrt_cost']
    long_vwaps[s_] = vwap.my_trades_analyses[s_]['true_long_vwap']
    shrt_vwaps[s_] = vwap.my_trades_analyses[s_]['true_shrt_vwap']
print('exit costs sorted low to high')
for itm in sorted([(e[0], round(e[1], 4)) for e in tlc.items()], key=lambda x: x[1])[-20:]:
    print('true long cost', itm)
print()
for itm in sorted([(e[0], round(e[1], 4)) for e in tsc.items()], key=lambda x: x[1])[-20:]:
    print('true shrt cost', itm)


In [ ]:
lst = [(e[0], round(e[1], 8), vwap.cm.last_price[e[0]], vwap.cm.last_price[e[0]] / (e[1] if e[1] else 10e-10)) for e in shrt_vwaps.items()]
print('shrts closest to filling:')
for e in sorted(lst, key=lambda x: x[3]):
    if e[0] not in vwap.do_shrt:
        continue
    
    line = f"{e[0]:<12} true shrt cost: {vwap.my_trades_analyses[e[0]]['true_shrt_cost']:.6f}"
    line += f" true shrt vwap: {e[1]:.8f} last price: {e[2]:.8f}"
    line += f" div {e[3]:.4f}"
    print(line)

    
    

In [ ]:
lst = [(e[0], round(e[1], 8), vwap.cm.last_price[e[0]], e[1] / vwap.cm.last_price[e[0]]) for e in long_vwaps.items()]
print('longs closest to filling:')
for e in sorted(lst, key=lambda x: x[3]):
    if e[0] not in vwap.do_long:
        continue
    
    line = f"{e[0]:<12} true long cost: {vwap.my_trades_analyses[e[0]]['true_long_cost']:.6f}"
    line += f" true long vwap: {e[1]:.8f} last price: {e[2]:.8f}"
    line += f" div {e[3]:.4f}"
    print(line)

    
    

In [ ]:
# past 24h vol per symbol
age_limit = vwap.cc.milliseconds() - 1000 * 60 * 60 * 24 * 7
vols = []
for s_ in vwap.my_trades:
    vols.append((round(sum([e['cost'] for e in vwap.my_trades[s_] if e['timestamp'] > age_limit]), 4), s_))
print(sum([e[0] for e in vols]))
sorted(vols)

In [ ]:
# overall performance analysis

In [ ]:
r = []
n_days = 30 # set n_days
start_ts = vwap.cc.milliseconds() - 1000 * 60 * 60 * 24 * n_days
for s_ in sorted(vwap.my_trades):
    sys.stdout.write(f'\r{s_}  ')
    sys.stdout.flush()
    mt = vwap.fetch_my_trades(s_)
    mtdf = pd.DataFrame(mt).set_index('timestamp')
    mtdf = mtdf[mtdf.index > start_ts]
    actual_n_days = (mtdf.index[-1] - mtdf.index[0]) / 1000 / 60 / 60 / 24
    buys = mtdf[mtdf.side == 'buy']
    sels = mtdf[mtdf.side == 'sell']
    bcost = buys.cost.sum()
    bamount = buys.amount.sum()
    bvwap = bcost / bamount
    scost = sels.cost.sum()
    samount = sels.amount.sum()
    svwap = scost / samount
    div_ = svwap / bvwap
    r.append({'bcost': bcost, 'scost': scost, 'svwap_bvwap': div_, 'actual_n_days': actual_n_days,
              'symbol': s_, 'cost': scost + bcost})

In [ ]:
rdf = pd.DataFrame(r)
rdf.loc[:,'vwg'] = (rdf.svwap_bvwap - 1) * rdf.cost / rdf.cost.sum()
rdf.sort_values('vwg')

In [ ]:
(rdf.svwap_bvwap * rdf.cost / rdf.cost.sum()).sum()

In [ ]:
# view balance logs

In [ ]:
with open(f'logs/binance/{user}/balance_margin.txt') as f:
    lines = f.readlines()

In [ ]:
bals = sorted([json.loads(line) for line in lines], key=lambda x: x['timestamp'])[2:]
ndays = (bals[-1]['timestamp'] - bals[0]['timestamp']) / 1000 / 60 / 60 / 24
print('n_days', ndays)

In [ ]:
len(bals)

In [ ]:
plt.plot([b['timestamp'] for b in bals], [b['BTC']['account_equity'] for b in bals])

In [ ]:
print('average daily gain')
(bals[-1]['BTC']['account_equity'] / bals[0]['BTC']['account_equity'])**(1 / ndays)

In [ ]:
'''
p * adg**ndays = r
adg**ndays = r / p
adg = (r / p)**(1 / ndays)
'''